In [51]:
import requests    
import json         
import pandas as pd 
import os

package = 'bevolkerung-nach-gemeinde-heimat-geschlecht-und-alter'

# Base url for the open data swiss API
base_url = 'https://opendata.swiss/api/3/action/package_show?id='

# Construct the url including package 
package_information_url = base_url + package

# HTTP request
package_information = requests.get(package_information_url)

# Use the json module to load CKAN's response into a dictionary
package_dict = json.loads(package_information.content)

# Check the contents of the response.
assert package_dict['success'] is True  
package_dict = package_dict['result']   
print(package_dict)             

{'license_title': None, 'maintainer': 'Statistisches Amt des Kantons Zürich, Data Shop', 'issued': '2020-02-03T14:26:00+01:00', 'title_for_slug': 'bevolkerung-nach-gemeinde-heimat-geschlecht-und-alter', 'qualified_relations': [], 'private': False, 'maintainer_email': 'datashop@statistik.zh.ch', 'num_tags': 13, 'contact_points': [{'email': 'datashop@statistik.zh.ch', 'name': 'Statistisches Amt des Kantons Zürich, Data Shop'}], 'keywords': {'fr': [], 'de': ['einwohnerzahl', 'geschlecht', 'altersgruppen', 'auslaender', 'gemeinden', 'heimat', 'altersklassen', 'bezirke', 'alter', 'bevoelkerungsstatistik', 'auslaenderinnen', 'altersstruktur', 'bevoelkerung'], 'en': [], 'it': []}, 'temporals': [{'start_date': '2010-01-01T00:00:00', 'end_date': '2023-12-31T23:59:59.999999'}], 'id': '3ef7cc79-6250-4637-9b5c-56b690ba8c7f', 'metadata_created': '2021-01-14T18:06:19.927677', 'documentation': [], 'conforms_to': [], 'metadata_modified': '2024-04-01T04:54:14.766529', 'author': None, 'author_email': No

In [52]:
# Get the url for the data from the dictionary
data_url = package_dict['resources'][0]['url']
print('Data url:' + data_url)

# Print the data format
data_format = package_dict['resources'][0]['format']
print('Data format:' + data_format)

Data url:https://www.web.statistik.zh.ch/ogd/daten/ressourcen/KTZH_00000254_00001282.csv
Data format:XLS


In [53]:
# Within the dictionary the data is not correctly marked as csv thats why we are saying xls but reading it as csv
xls = ['XLS']
if any(s in data_format for s in xls):
    df = pd.read_csv(data_url,sep=';')
else:
    print('Sorry, the data format is not supported')
df

,jahr,gemeinde_bfs_nr,gemeinde,bezirk_code,bezirk,region_code,region_name,heimat_code,heimat,geschlecht_code,geschlecht,einjahresaltersklasse,anzahl
0,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,0,8
1,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,1,9
2,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,2,7
3,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,3,13
4,2010,1,Aeugst am Albis,101,Affoltern,105,Knonaueramt,1,Schweiz,1,Mann,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
736184,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,84,1
736185,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,85,2
736186,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,87,1
736187,2023,298,Wiesendangen,110,Winterthur,109,Winterthur und Umgebung,2,Ausland,2,Frau,88,1


In [54]:
df.drop(['bezirk_code','bezirk','region_code','region_name','heimat_code','geschlecht_code','geschlecht','heimat','gemeinde_bfs_nr'], axis=1, inplace=True)
df

,jahr,gemeinde,einjahresaltersklasse,anzahl
0,2010,Aeugst am Albis,0,8
1,2010,Aeugst am Albis,1,9
2,2010,Aeugst am Albis,2,7
3,2010,Aeugst am Albis,3,13
4,2010,Aeugst am Albis,4,5
...,...,...,...,...
736184,2023,Wiesendangen,84,1
736185,2023,Wiesendangen,85,2
736186,2023,Wiesendangen,87,1
736187,2023,Wiesendangen,88,1


In [55]:
# Vereinheitlichen der Gemeindenamen
df['gemeinde'] = df['gemeinde'].str.replace('Aesch (ZH)', 'Aesch', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Benken (ZH)', 'Benken', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Birmensdorf (ZH)', 'Birmensdorf', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Buchs (ZH)', 'Buchs', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Erlenbach (ZH)', 'Erlenbach', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Gossau (ZH)', 'Gossau', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Kilchberg (ZH)', 'Kilchberg', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Küsnacht (ZH)', 'Küsnacht', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Rickenbach (ZH)', 'Rickenbach', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Rüti (ZH)', 'Rüti', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Schlatt (ZH)', 'Schlatt', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Wald (ZH)', 'Wald', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Weiningen (ZH)', 'Weiningen', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Wetzikon (ZH)', 'Wetzikon', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Wil (ZH)', 'Wil', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Zell (ZH)', 'Zell', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Aeugst am Albis', 'Aeugst', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Affoltern am Albis', 'Affoltern', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Berg am Irchel', 'Berg', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Buch am Irchel', 'Buch', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Ellikon an der Thur', 'Ellikon', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Hausen am Albis', 'Hausen', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Kappel am Albis', 'Kappel', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Langnau am Albis', 'Langnau', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Oetwil am See', 'Oetwil', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Thalheim an der Thur', 'Thalheim', regex=False)
df['gemeinde'] = df['gemeinde'].str.replace('Uetikon am See', 'Uetikon', regex=False)


df

,jahr,gemeinde,einjahresaltersklasse,anzahl
0,2010,Aeugst,0,8
1,2010,Aeugst,1,9
2,2010,Aeugst,2,7
3,2010,Aeugst,3,13
4,2010,Aeugst,4,5
...,...,...,...,...
736184,2023,Wiesendangen,84,1
736185,2023,Wiesendangen,85,2
736186,2023,Wiesendangen,87,1
736187,2023,Wiesendangen,88,1


In [56]:
import pandas as pd

altersklassen = {
    '0-14 Jahre alt': range(0, 15),
    '15-29 Jahre alt': range(15, 30),
    '30-44 Jahre alt': range(30, 45),
    '45-59 Jahre alt': range(45, 60),
    '60-74 Jahre alt': range(60, 75),
    '75-89 Jahre alt': range(75, 90),
    '90-104 Jahre alt': range(90, 105)
}

# Funktion zur Aggregation der Altersklassen
def summiere_altersklassen(gruppe):
    summen = {}
    for altersklasse, bereich in altersklassen.items():
        summen[altersklasse] = gruppe[gruppe['einjahresaltersklasse'].isin(bereich)]['anzahl'].sum()
    return pd.Series(summen, dtype='int64')

# Gruppiere den DataFrame nach Gemeinde und Jahr und wende die Funktion auf jede Gruppe an
summierte_df = df.groupby(['gemeinde', 'jahr'], as_index=False).apply(summiere_altersklassen)

# Ergebnis anzeigen
print(summierte_df)


      gemeinde  jahr  0-14 Jahre alt  15-29 Jahre alt  30-44 Jahre alt  \
0     Adliswil  2010            2348             2907             3778   
1     Adliswil  2011            2510             3087             4130   
2     Adliswil  2012            2582             3159             4367   
3     Adliswil  2013            2673             3060             4476   
4     Adliswil  2014            2786             2993             4503   
...        ...   ...             ...              ...              ...   
2235    Zürich  2019           58927            72875           125728   
2236    Zürich  2020           59578            72433           126541   
2237    Zürich  2021           59876            73173           126052   
2238    Zürich  2022           59694            75261           127463   
2239    Zürich  2023           59958            76773           128838   

      45-59 Jahre alt  60-74 Jahre alt  75-89 Jahre alt  90-104 Jahre alt  
0                3389             2

/var/folders/lt/9lkssqds44vcn4hwlh7dd2wc0000gn/T/ipykernel_7325/1620022716.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summierte_df = df.groupby(['gemeinde', 'jahr'], as_index=False).apply(summiere_altersklassen)


In [57]:
current_directory = os.getcwd()

file_path = os.path.join(current_directory, "Bevoelkerung_KantonZurich_2010_2023.csv")
summierte_df.to_csv(file_path, index=False)

print(f"The DataFrame has been saved to '{file_path}'.")

The DataFrame has been saved to '/Users/natalievintonjak/Documents/Data Science Project/Bevoelkerung_KantonZurich_2010_2023.csv'.
